# Employing spherical trigonometry or geodesy to accurately determine   
# the distance and bearing between two positions of the Sentinel 2A satellite.

Within this notebook, we will meticulously apply the principles of spherical trigonometry to accurately calculate   
the distance and cardinal direction between two specified coordinates representing the Sentinel 2A satellite's position.   
This rigorous approach will enable us to precisely determine the satellite's spatial displacement and orientation.

### Install Skyfield to get Geographical position in real time.

In [1]:
!pip install skyfield

In [2]:
from skyfield.api import load, EarthSatellite, wgs84, utc
from datetime import datetime, timedelta
import math

### Mathematical formula  
For two points with coordinates (𝑙𝑎𝑡1,𝑙𝑜𝑛1) and (𝑙𝑎𝑡2,𝑙𝑜𝑛2) in decimal degrees,  
the formula to calculate the bearing (direction in degrees) from the first point to the second is:  
\


Where:  

θ is the bearing (in radians).  
ϕ1 and 𝜙2 are the latitudes of the points in radians.  
λ1 and 𝜆2 are the longitudes of the points in radians.  
Δ𝜆=𝜆2−𝜆1 is the difference in longitudes in radians.  

Finally, to convert the angle to degrees and ensure that the result is in the range of 0° to 360°:

##### Function to calculate the direction between two geographic coordinates

In [3]:
def get_direction(lat1, lon1, lat2, lon2):
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    delta_lon = lon2_rad - lon1_rad

    x = math.sin(delta_lon) * math.cos(lat2_rad)
    y = math.cos(lat1_rad) * math.sin(lat2_rad) - (math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon))

    angle_radians = math.atan2(x, y)
    angle_degrees  = (math.degrees(angle_radians) + 360) % 360

    return angle_degrees 

##### TLE of Sentinel-2A satellite

I obtained the TLE information for satellite Sentinel 2A from https://www.n2yo.com/satellite/?s=40697.  
This website is a satellite tracking service that provides TLE data and related information for various satellites.  
By accessing this specific URL, you retrieved the current TLE for satellite 40697, which can be used to determine its orbital characteristics, predict its position, and calculate future passes. Such services typically update their TLE data regularly from authoritative sources to maintain accuracy in satellite tracking and predictions.

In [4]:
tle_line1 = '1 40697U 15028A   24263.15857056  .00000459  00000-0  19166-3 0  9990'
tle_line2 = '2 40697  98.5710 336.0659 0001038  93.3847 266.7455 14.30812527482783'

##### Create the Sentinel-2A satellite using TLE data

In [5]:
sentinel2a = EarthSatellite(tle_line1, tle_line2, 'Sentinel-2A', load.timescale())

##### Load time table

In [6]:
ts = load.timescale()

In [7]:
# Define the current time
t1 = ts.now()

In [8]:
# Define a future time (e.g. 60 seconds later)
t2 = ts.utc((datetime.utcnow() + timedelta(seconds=60)).replace(tzinfo=utc))

##### Get the satellite position at time t1

In [9]:
geocentric1 = sentinel2a.at(t1)
subpoint1 = wgs84.subpoint(geocentric1)
lat1 = subpoint1.latitude.degrees
lon1 = subpoint1.longitude.degrees

##### Get the satellite position at time t2

In [10]:
geocentric2 = sentinel2a.at(t2)
subpoint2 = wgs84.subpoint(geocentric2)
lat2 = subpoint2.latitude.degrees
lon2 = subpoint2.longitude.degrees

### Function to determine the cardinal direction from degrees

In [11]:
def get_cardinal_direction(direccion_grados):
    if (direccion_grados >= 337.5 or direccion_grados < 22.5):
        return "Norte"
    elif (22.5 <= direccion_grados < 67.5):
        return "Noreste"
    elif (67.5 <= direccion_grados < 112.5):
        return "Este"
    elif (112.5 <= direccion_grados < 157.5):
        return "Sureste"
    elif (157.5 <= direccion_grados < 202.5):
        return "Sur"
    elif (202.5 <= direccion_grados < 247.5):
        return "Suroeste"
    elif (247.5 <= direccion_grados < 292.5):
        return "Oeste"
    elif (292.5 <= direccion_grados < 337.5):
        return "Noroeste"

# Calculate the direction of satellite movement
movement_directioon = get_direction(lat1, lon1, lat2, lon2)

# Get the cardinal direction
cardinal_direction = get_cardinal_direction(movement_directioon)

print(f"Starting position: Latitude {lat1}, Longitude {lon1}")
print(f"Position 60 seconds later: Latitude {lat2}, Longitude {lon2}")
print(f"Satellire movement direction: {movement_directioon} degrees")
print(f"Cardinal direction: {cardinal_direction}")

Starting position: Latitude 9.583948615698084, Longitude 117.186626289487
Position 60 seconds later: Latitude 13.139744899234515, Longitude 116.38153640204476
Satellire movement direction: 347.56166905496974 degrees
Cardinal direction: Norte


In [12]:
# Install folium to show satellite position on interactive map
!pip install folium

In [13]:
import folium
# Create the map centered on the first satellite position
mapa = folium.Map(location=[lat1, lon1], zoom_start=4)

# Add a marker for the initial satellite position
folium.Marker([lat1, lon1], popup=f"Starting position: Latitude {lat1}, Longitude {lon1}").add_to(mapa)

# Add an arrow indicating the direction of movement
folium.PolyLine(locations=[[lat1, lon1], [lat2, lon2]], color='blue', weight=2.5, opacity=1).add_to(mapa)

# Add a marker for the future satellite position (after 60 seconds)
folium.Marker([lat2, lon2], popup=f"Future position: Latitude {lat2}, Longitude {lon2}").add_to(mapa)

# Show map
mapa